In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=7f059646bb2f082d28abfd49cb78d2e140119236c5914ca784876f06775fe1cc
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [16]:
import pyspark
import os

from datetime import datetime
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import StructType, StructField, StringType, DateType

In [4]:
from google.colab import drive
drive.mount('/content/drive')
path = 'drive/MyDrive/MyData/'

Mounted at /content/drive


In [13]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.13.0 pyspark-shell'

app_name = "lb2"

sc = SparkSession.builder.appName(app_name).master("local[*]").getOrCreate()
sc

In [6]:
data = sc.read.csv(path + "/bigdataset/data/programming-languages.csv")

In [7]:
languages = [str(x[0]) for x in data.collect()]

languages[:10]

['name',
 'A# .NET',
 'A# (Axiom)',
 'A-0 System',
 'A+',
 'A++',
 'ABAP',
 'ABC',
 'ABC ALGOL',
 'ABSET']

In [8]:
posts_samples = sc.read.format("xml").options(rowTag="row").load(path + '/bigdataset/data/posts_sample.xml')

posts_samples.first()

Row(_AcceptedAnswerId=7, _AnswerCount=13, _Body="<p>I want to use a track-bar to change a form's opacity.</p>\n\n<p>This is my code:</p>\n\n<pre><code>decimal trans = trackBar1.Value / 5000;\nthis.Opacity = trans;\n</code></pre>\n\n<p>When I build the application, it gives the following error:</p>\n\n<blockquote>\n  <p>Cannot implicitly convert type <code>'decimal'</code> to <code>'double'</code></p>\n</blockquote>\n\n<p>I tried using <code>trans</code> and <code>double</code> but then the control doesn't work. This code worked fine in a past VB.NET project.</p>\n", _ClosedDate=None, _CommentCount=2, _CommunityOwnedDate=datetime.datetime(2012, 10, 31, 16, 42, 47, 213000), _CreationDate=datetime.datetime(2008, 7, 31, 21, 42, 52, 667000), _FavoriteCount=48, _Id=4, _LastActivityDate=datetime.datetime(2019, 7, 19, 1, 39, 54, 173000), _LastEditDate=datetime.datetime(2019, 7, 19, 1, 39, 54, 173000), _LastEditorDisplayName='Rich B', _LastEditorUserId=3641067, _OwnerDisplayName=None, _OwnerUse

Функция для обработки данных

In [9]:
def findLanguages(x):
  tag = None

# Ищем вхождение названия ЯП в строке в нижнем регистре
  for language in languages:
    if "<" + language.lower() + ">" in x._Tags.lower():
      tag = language
      break

  if tag is None:
    return None

  return (x._Id, tag)

In [10]:
def isDateValid(x, year):
  start = datetime(year=year, month=1, day=1)
  end = datetime(year=year, month=12, day=31)

  CreationDate = x._CreationDate

  return CreationDate >= start and CreationDate <= end

Обработка

In [11]:
result = {}

for year in range(2010, 2020):
  result[year] = posts_samples.rdd\
      .filter(lambda x: x._Tags is not None and isDateValid(x, year))\
      .map(findLanguages)\
      .filter(lambda x: x is not None)\
      .keyBy(lambda x: x[1])\
      .aggregateByKey(
          0,
          lambda x, y: x + 1,
          lambda x1, x2: x1 + x2,
      )\
      .sortBy(lambda x: x[1], ascending=False)\
      .toDF()

  result[year] = result[year].select(col("_1").alias("Language"),
                                                 col("_2").alias(f"Mentions_{year}")).limit(10)
  result[year].show()

+-----------+-------------+
|   Language|Mentions_2010|
+-----------+-------------+
|       Java|           52|
| JavaScript|           44|
|        PHP|           42|
|     Python|           25|
|Objective-C|           22|
|          C|           20|
|       Ruby|           11|
|     Delphi|            7|
|          R|            3|
|       Bash|            3|
+-----------+-------------+

+-----------+-------------+
|   Language|Mentions_2011|
+-----------+-------------+
|        PHP|           97|
|       Java|           92|
| JavaScript|           82|
|     Python|           35|
|Objective-C|           33|
|          C|           24|
|       Ruby|           17|
|     Delphi|            8|
|       Perl|            8|
|       Bash|            7|
+-----------+-------------+

+-----------+-------------+
|   Language|Mentions_2012|
+-----------+-------------+
|        PHP|          136|
| JavaScript|          129|
|       Java|          124|
|     Python|           65|
|Objective-C|     

In [19]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName(app_name).getOrCreate()

# сохраняем результаты в Parquet формате
for year in range(2010, 2020):
    result[year].write.mode("overwrite").parquet(path + f"result_{year}.parquet")